In [27]:
import adagram
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
punct = punctuation+'«»—…“”*№–'
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report



def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
    words = tokenize(text)
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

In [2]:
!pip install Cython numpy
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-n161j5d9
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-n161j5d9
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=460304 sha256=99b45ae596edb82e4f221fe16cea7a7e7201d3bc32cb6be85d8884fbf091ac7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-gsyp52t6/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram


In [6]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# модель обученная на большом корпусе (острожно 1.5 гб)
!curl 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib' --output all.a010.p10.d300.w5.m100.nonorm.slim.joblib
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  45.3M      0  0:00:30  0:00:30 --:--:-- 47.4M


## Задание № 1. Протестировать адаграм в определении перефразирования

In [9]:
import pandas as pd

In [18]:
%cd /

/


In [21]:
%cd /content/drive/My\ Drive/WSI


/content/drive/My Drive/WSI


In [23]:
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [28]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

Векторизуйте пары текстов с помощью Адаграма, обучите любую модель и оцените качество (кросс-валидацией). 

За основу возьмите код из предыдущего семинара/домашки, только в функции get_embedding вам нужно выбирать вектор нужного значения (импользуйте model.disambiguate и model.sense_vector). Отдельные векторы усредните как и в предыдущем семинаре.

Для вытаскивания пар (целевое слово, контекстые слова) вам нужно будет написать специальную функцию.

In [29]:
# проверяте на списке из чисел, чтобы было удобно дебажить
words = [0,1,2,3,4,5,6,7,8,9]

def get_words_in_context(words, window=3):
    words_in_context = []
    for idx, word in enumerate(words):
        context = []
        for ctx_idx, ctx_word in enumerate(words):
            residual = abs(idx - ctx_idx)
            if residual > 0 and residual <= window:
                context.append(ctx_word)
        words_in_context.append([word, context])
    return words_in_context

In [30]:
get_words_in_context(words)

[[0, [1, 2, 3]],
 [1, [0, 2, 3, 4]],
 [2, [0, 1, 3, 4, 5]],
 [3, [0, 1, 2, 4, 5, 6]],
 [4, [1, 2, 3, 5, 6, 7]],
 [5, [2, 3, 4, 6, 7, 8]],
 [6, [3, 4, 5, 7, 8, 9]],
 [7, [4, 5, 6, 8, 9]],
 [8, [5, 6, 7, 9]],
 [9, [6, 7, 8]]]

Когда получиться такой же результат, добавьте эту функцию в get_embedding. Проходите циклом по результату работы get_words_in_context и поставляйте каждый элемент-список в model.disambiguate.

In [38]:
def get_embedding_adagram(text, model, window, dim):
    
    word2context = get_words_in_context(text, window)
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i, (word, context) in enumerate(word2context):
        
        try:
            sense_id = model.disambiguate(word, context).argmax()
            vectors[i] = model.sense_vector(word, sense_id)
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector


In [39]:
dim = vm.dim
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding_adagram(text, vm, 3, dim)

for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding_adagram(text, vm, 3, dim)

In [40]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [42]:
y = data['label'].values
print(y.shape)
train_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)
clf = RandomForestClassifier(n_estimators=150, max_depth=6, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

(7227,)
              precision    recall  f1-score   support

          -1       0.53      0.51      0.52       629
           0       0.48      0.50      0.49       737
           1       0.38      0.38      0.38       441

    accuracy                           0.47      1807
   macro avg       0.46      0.46      0.46      1807
weighted avg       0.47      0.47      0.47      1807



## WordNet

Выводить значения просто из текста тяжело, поэтому можно попробовать воспользоваться словарями (wsi сделанное людьми). Для русского придется парсить сайты словарей, а для английского можно воспользоваться WordNet'ом (https://wordnet.princeton.edu/), который есть в nltk.

WordNet - это лексическая база данных, где существительные, прилагательные и глаголы английского сгруппированы по значению. К тому же между ними установлены связи (гипонимия, гипоронимия, миронимия и т.п.).

In [43]:
# запустите если не установлен ворднет
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Задание 2. Реализовать алгоритм Леска и проверить его на реальном датасете

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [44]:
def lesk(word, sentence):
    """Ваш код тут"""
    sentence = set(sentence)
    bestsense = 0
    maxoverlap = 0
    synsets = wn.synsets(word)
    for i, syns in enumerate(synsets):
        definition = set(syns.definition().lower().split())
        overlap = len(sentence & definition)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
    return bestsense

In [45]:
lesk('day', 'some point or period in time'.split())

1

Работать функция должна как-то так:

In [46]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time'.split()) # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

**Проверьте насколько хорошо работает такой метод на реальном датасете.** http://lcl.uniroma1.it/wsdeval/ - большой фреймворк для оценки WSD. Там много данных и я взял кусочек, чтобы не было проблем с памятью

In [47]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

In [50]:
corpus_wsd[0:1]

[[['', 'how', 'How'],
  ['long%3:00:02::', 'long', 'long'],
  ['', 'have', 'has'],
  ['', 'it', 'it'],
  ['be%2:42:03::', 'be', 'been'],
  ['', 'since', 'since'],
  ['', 'you', 'you'],
  ['review%2:31:00::', 'review', 'reviewed'],
  ['', 'the', 'the'],
  ['objective%1:09:00::', 'objective', 'objectives'],
  ['', 'of', 'of'],
  ['', 'you', 'your'],
  ['benefit%1:21:00::', 'benefit', 'benefit'],
  ['', 'and', 'and'],
  ['service%1:04:07::', 'service', 'service'],
  ['program%1:09:01::', 'program', 'program'],
  ['', '?', '?']]]

In [70]:
from copy import copy
correct_count = 0
word_count = 0
tokens = []
for sent in corpus_wsd:
    
    for word in sent:
      if len(word) == 3:
        tokens.append(word[1])
      else:
        continue
      context = copy(sent)
      context.remove(word)
      context = [word[1] for word in context ]
      word_count += 1
      if word[0] != '':
          real_synset = wn.lemma_from_key(word[0]).synset()
          predicted_synset = wn.synsets(word[1])[lesk(word[1], context)]
          if predicted_synset == real_synset:
              correct_count += 1

print(correct_count / word_count)
        
                    

0.06498187328413578


**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

### Дополнительный балл

Если хотите заработать дополнительный балл, попробуйте улучшить алгоритм Леска любым способом (например, использовать расстояние редактирования вместо пересечения или даже вставить машинное обучение)